In [150]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler

In [18]:
file_1h = './train_public/1h'
file_30min = './train_public/30min'
file_15min = './train_public/15min'
file_5min = './train_public/5min'

csvfiles_1h = [file_1h + '/' + file for file in os.listdir(file_1h)]
csvfiles_1h.sort()
csvfiles_1h

['./train_public/1h/L03.B02_1H.csv',
 './train_public/1h/L06.B01_1H.csv',
 './train_public/1h/L09.B01_1H.csv',
 './train_public/1h/L10.B01_1H.csv',
 './train_public/1h/L14.B01_1H.csv',
 './train_public/1h/L14.B02_1H.csv',
 './train_public/1h/L14.B03_1H.csv',
 './train_public/1h/L14.B04_1H.csv',
 './train_public/1h/L14.B05_1H.csv']

<img src='figures/data_description.png' width=500, height=450>

In [43]:
l03b02_1h = pd.read_csv(csvfiles_1h[0])
l06b01_1h = pd.read_csv(csvfiles_1h[1])
l09b01_1h = pd.read_csv(csvfiles_1h[2])
l10b01_1h = pd.read_csv(csvfiles_1h[3])
l14b01_1h = pd.read_csv(csvfiles_1h[4])
l14b02_1h = pd.read_csv(csvfiles_1h[5])
l14b03_1h = pd.read_csv(csvfiles_1h[6])
l14b04_1h = pd.read_csv(csvfiles_1h[7])
l14b05_1h = pd.read_csv(csvfiles_1h[8])

data_1h = {'l03b02_1h': l03b02_1h,
           'l06b01_1h': l06b01_1h,  
           'l09b01_1h': l09b01_1h,
           'l10b01_1h': l10b01_1h, 
           'l14b01_1h': l14b01_1h, 
           'l14b02_1h': l14b02_1h, 
           'l14b03_1h': l14b03_1h, 
           'l14b04_1h': l14b04_1h, 
           'l14b05_1h': l14b05_1h
    }

l03b02_1h.head()
l06b01_1h.head()

for key, value in data_1h.items():
    print(key, value.shape)

l03b02_1h (8400, 2)
l06b01_1h (7674, 3)
l09b01_1h (8287, 3)
l10b01_1h (13867, 2)
l14b01_1h (8760, 2)
l14b02_1h (8760, 2)
l14b03_1h (8760, 2)
l14b04_1h (8760, 2)
l14b05_1h (8760, 2)


In [ ]:
# Function to disaggregate heating and cooling using NMF
def disaggregate_energy(file_path, weather_file_path, output_file_path):
    # Load the building energy data and weather data
    energy_data = pd.read_csv(file_path)
    weather_data = pd.read_csv(weather_file_path)

    # Merge datasets on timestamp
    data = pd.merge(energy_data, weather_data, on='timestamp')

    # Select relevant features for NMF (including weather data)
    features = ['main_meter(kW)', 'air_temperature_at_2m(deg_C)', 'relative_humidity_at_2m(%)']

    # Normalize the data to ensure non-negativity
    scaler = MinMaxScaler()
    X_normalized = scaler.fit_transform(data[features])

    # Apply NMF to the combined dataset
    nmf_model = NMF(n_components=2, init='random', random_state=42, max_iter=1000)
    W = nmf_model.fit_transform(X_normalized)
    H = nmf_model.components_

    # Denormalize the results back to the original scale
    W_denormalized = scaler.inverse_transform(W)

    # Extract the heating and cooling loads (assuming the first component corresponds to heating)
    heating_and_cooling_load = W_denormalized[:, 0]  # Assuming the first component is heating/cooling

    # Prepare the output dataframe
    output_data = pd.DataFrame({
        'timestamp': energy_data['timestamp'],
        'heating_and_cooling_load(kW)': heating_and_cooling_load
    })

    # Save the disaggregated data to a new CSV file
    output_data.to_csv(output_file_path, index=False)

# Directory paths for input and output
input_dir = 'path_to_input_directory'  # Replace with the path to your input directory
output_dir = 'path_to_output_directory'  # Replace with the path to your output directory
weather_dir = 'path_to_weather_directory'  # Replace with the path to your weather data directory

# Process each building's data
for i in range(1, 10):  # Assuming the files are named in a sequence, e.g., building1.csv, building2.csv, ...
    input_file = os.path.join(input_dir, f'building{i}.csv')
    weather_file = os.path.join(weather_dir, f'weather_building{i}.csv')  # Assuming each building has its own weather file
    output_file = os.path.join(output_dir, f'building{i}_heating_cooling.csv')

    # Disaggregate and save the results
    disaggregate_energy(input_file, weather_file, output_file)


In [31]:
l03_weather = pd.read_csv('train_public/weather/L03_weather_train.csv')
l06_weather = pd.read_csv('train_public/weather/L06_weather_train.csv')
l09_weather = pd.read_csv('train_public/weather/L09_weather_train.csv')
l10_weather = pd.read_csv('train_public/weather/L10_weather_train.csv')
l14_weather = pd.read_csv('train_public/weather/L14_weather_train.csv')

data_weather = {'l03': l03_weather,
                'l06': l06_weather,
                'l09': l09_weather,
                'l10': l10_weather,
                'l14': l14_weather
}

l03_weather.head()

,timestamp,air_temperature_at_2m(deg_C),relative_humidity_at_2m(%),direct_solar_radiation(W/m^2),diffuse_solar_radiation(W/m^2),wind_speed_at_10m(km/h),wind_direction_at_10m(deg)
0,2020-12-01 00:00:00+00:00,3.444376,97.016089,0.026658,-0.054859,15.924282,175.974536
1,2020-12-01 01:00:00+00:00,3.662187,96.964139,-0.022419,0.039787,13.008325,174.044967
2,2020-12-01 02:00:00+00:00,3.849796,97.042752,0.018184,-0.019089,9.631573,167.017767
3,2020-12-01 03:00:00+00:00,4.047407,97.044695,0.054901,-0.016765,6.825520,148.045236
4,2020-12-01 04:00:00+00:00,4.184971,97.035953,-0.022905,-0.012460,6.036958,114.996164


In [27]:
l03_weather.tail()

,timestamp,air_temperature_at_2m(deg_C),relative_humidity_at_2m(%),direct_solar_radiation(W/m^2),diffuse_solar_radiation(W/m^2),wind_speed_at_10m(km/h),wind_direction_at_10m(deg)
8395,2021-11-15 19:00:00+00:00,6.910766,81.985465,-0.036131,0.038543,6.930305,137.044424
8396,2021-11-15 20:00:00+00:00,6.753975,81.976869,0.015393,0.038366,7.456723,144.981432
8397,2021-11-15 21:00:00+00:00,6.582176,82.985219,0.014573,0.001191,7.893027,149.998625
8398,2021-11-15 22:00:00+00:00,6.631711,83.958208,-0.020878,-0.013877,6.360064,142.022950
8399,2021-11-15 23:00:00+00:00,6.427195,84.972944,0.038590,-0.000835,6.216857,143.977515


In [36]:
for key, value in data_weather.items():
    print(key, value.shape)

l03 (8400, 7)
l06 (7704, 7)
l09 (8760, 7)
l10 (13872, 7)
l14 (8760, 7)


In [44]:
for key, value in data_1h.items():
    print(key, value.shape)

l03b02_1h (8400, 2)
l06b01_1h (7674, 3)
l09b01_1h (8287, 3)
l10b01_1h (13867, 2)
l14b01_1h (8760, 2)
l14b02_1h (8760, 2)
l14b03_1h (8760, 2)
l14b04_1h (8760, 2)
l14b05_1h (8760, 2)


In [47]:
# Function to clean weather data based on location
def clean_weather_data_by_location(file_path, location):
    # Load the weather data
    weather_data = pd.read_csv(file_path)

    # Location-specific temperature ranges
    temp_ranges = {
        'l03': (-10, 35),  # Denmark
        'l06': (-30, 30),  # Norway
        'l09': (-5, 45),   # Australia
        'l10': (-30, 30),  # Norway
        'l14': (-30, 30)   # Norway
    }

    # Get the appropriate temperature range for the location
    temp_min, temp_max = temp_ranges[location]

    # Clean the temperature data
    weather_data['air_temperature_at_2m(deg_C)'] = weather_data['air_temperature_at_2m(deg_C)'].clip(temp_min, temp_max)

    # Set negative values in solar radiation columns to 0
    weather_data['direct_solar_radiation(W/m^2)'] = weather_data['direct_solar_radiation(W/m^2)'].clip(lower=0)
    weather_data['diffuse_solar_radiation(W/m^2)'] = weather_data['diffuse_solar_radiation(W/m^2)'].clip(lower=0)

    # Clip humidity between 0% and 100%
    weather_data['relative_humidity_at_2m(%)'] = weather_data['relative_humidity_at_2m(%)'].clip(0, 100)

    # Set negative wind speeds to 0
    weather_data['wind_speed_at_10m(km/h)'] = weather_data['wind_speed_at_10m(km/h)'].clip(lower=0)

    # Optionally, you could also set a maximum wind speed threshold
    weather_data['wind_speed_at_10m(km/h)'] = weather_data['wind_speed_at_10m(km/h)'].clip(0, 150)

    # Save the cleaned data back to the CSV file (or a new file)
    cleaned_file_path = file_path.replace('.csv', f'_{location}_cleaned.csv')
    weather_data.to_csv(cleaned_file_path, index=False)

    return cleaned_file_path

# Example of applying the cleaning function to all weather datasets
weather_files = {
    'l03': 'train_public/weather/L03_weather_train.csv',
    'l06': 'train_public/weather/L06_weather_train.csv',
    'l09': 'train_public/weather/L09_weather_train.csv',
    'l10': 'train_public/weather/L10_weather_train.csv',
    'l14': 'train_public/weather/L14_weather_train.csv'
}

for location, weather_file in weather_files.items():
    cleaned_file = clean_weather_data_by_location(weather_file, location)
    print(f"Cleaned data for {location} saved to: {cleaned_file}")

Cleaned data for l03 saved to: train_public/weather/L03_weather_train_l03_cleaned.csv
Cleaned data for l06 saved to: train_public/weather/L06_weather_train_l06_cleaned.csv
Cleaned data for l09 saved to: train_public/weather/L09_weather_train_l09_cleaned.csv
Cleaned data for l10 saved to: train_public/weather/L10_weather_train_l10_cleaned.csv
Cleaned data for l14 saved to: train_public/weather/L14_weather_train_l14_cleaned.csv


In [69]:
j = 0
for i in l06b01_1h.iloc[:, 1]:
    if i < 0:
        j = j + 1
print(j)

236


In [68]:
l06b01_1h[3100:3150]

,timestamp,main_meter(kW),PV_battery_system(kW)
3100,2021-08-08 04:00:00+00:00,11.628814,0.805000
3101,2021-08-08 05:00:00+00:00,8.690321,4.924546
3102,2021-08-08 06:00:00+00:00,-1.262319,14.790002
3103,2021-08-08 07:00:00+00:00,-10.246774,23.699652
3104,2021-08-08 08:00:00+00:00,-20.815252,33.356487
3105,2021-08-08 09:00:00+00:00,-15.234246,32.414921
3106,2021-08-08 10:00:00+00:00,-22.268257,37.618954
3107,2021-08-08 11:00:00+00:00,-17.697014,32.018768
3108,2021-08-08 12:00:00+00:00,-10.589552,25.286966
3109,2021-08-08 13:00:00+00:00,-8.533872,22.153507


In [72]:
l09b01_1h.tail(50)

,timestamp,main_meter(kW),PV_battery_system(kW)
8237,2021-12-29 22:00:00+00:00,-110.583333,188.416667
8238,2021-12-29 23:00:00+00:00,-97.583333,161.250000
8239,2021-12-30 00:00:00+00:00,-167.333333,240.250000
8240,2021-12-30 01:00:00+00:00,-218.692308,295.846154
8241,2021-12-30 02:00:00+00:00,-216.083333,293.833333
8242,2021-12-30 03:00:00+00:00,-205.166667,281.833333
8243,2021-12-30 04:00:00+00:00,-178.083333,255.916667
8244,2021-12-30 05:00:00+00:00,-132.833333,211.000000
8245,2021-12-30 06:00:00+00:00,-78.750000,152.583333
8246,2021-12-30 07:00:00+00:00,-28.000000,101.666667


In [73]:
j = 0
for i in l09b01_1h.iloc[:, 2]:
    if i < 0:
        j = j + 1
print(j)

3375


In [74]:
k = 0
for i, j in zip(l09b01_1h.iloc[:, 1], l09b01_1h.iloc[:, 2]):
    if i < 0 and j < 0:
        k = k + 1
print(k)

0


In [80]:
l03_weather.head()

,timestamp,air_temperature_at_2m(deg_C),relative_humidity_at_2m(%),direct_solar_radiation(W/m^2),diffuse_solar_radiation(W/m^2),wind_speed_at_10m(km/h),wind_direction_at_10m(deg)
0,2020-12-01 00:00:00+00:00,3.444376,97.016089,0.026658,-0.054859,15.924282,175.974536
1,2020-12-01 01:00:00+00:00,3.662187,96.964139,-0.022419,0.039787,13.008325,174.044967
2,2020-12-01 02:00:00+00:00,3.849796,97.042752,0.018184,-0.019089,9.631573,167.017767
3,2020-12-01 03:00:00+00:00,4.047407,97.044695,0.054901,-0.016765,6.825520,148.045236
4,2020-12-01 04:00:00+00:00,4.184971,97.035953,-0.022905,-0.012460,6.036958,114.996164


In [78]:
l03b02_1h

,timestamp,main_meter(kW)
0,2020-12-01 00:00:00+00:00,69.778595
1,2020-12-01 01:00:00+00:00,52.449440
2,2020-12-01 02:00:00+00:00,72.574860
3,2020-12-01 03:00:00+00:00,67.813957
4,2020-12-01 04:00:00+00:00,65.743805
...,...,...
8395,2021-11-15 19:00:00+00:00,148.864655
8396,2021-11-15 20:00:00+00:00,151.322693
8397,2021-11-15 21:00:00+00:00,120.674629
8398,2021-11-15 22:00:00+00:00,113.046677


In [81]:
l06b01_1h.head()

,timestamp,main_meter(kW),PV_battery_system(kW)
0,2021-04-01 00:00:00+00:00,62.873325,NaN
1,2021-04-01 01:00:00+00:00,66.489815,0.0
2,2021-04-01 02:00:00+00:00,65.625000,NaN
3,2021-04-01 03:00:00+00:00,67.636673,NaN
4,2021-04-01 04:00:00+00:00,65.049164,NaN


In [98]:
# Function to clean a building energy consumption dataset
def clean_building_energy_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # 1. Check for daylight saving time or duplicated timestamps
    duplicated_timestamps = data['timestamp'][data['timestamp'].duplicated()]
    if not duplicated_timestamps.empty:
        print(f"Duplicate or suspicious timestamps found in {file_path}:")
        print(duplicated_timestamps)

    # 2. Split the "timestamp" column into "date" and "time"
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['date'] = data['timestamp'].dt.date
    data['time'] = data['timestamp'].dt.time

    # Drop the old "timestamp" column
    data = data.drop(columns=['timestamp'])

    # 3. Calculate "energy_consumption(kW)"
    if 'PV_battery_system(kW)' in data.columns:
        data['energy_consumption(kW)'] = data['main_meter(kW)'] + data['PV_battery_system(kW)']
        data = data.drop(columns=['main_meter(kW)', 'PV_battery_system(kW)'])
    else:
        data = data.rename(columns={'main_meter(kW)': 'energy_consumption(kW)'})

    # Save the cleaned data back to a new CSV file
    cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')
    data.to_csv(cleaned_file_path, index=False)

    return cleaned_file_path

# List of building energy files
building_files = [
 './train_public/1h/L03.B02_1H.csv',
 './train_public/1h/L06.B01_1H.csv',
 './train_public/1h/L09.B01_1H.csv',
 './train_public/1h/L10.B01_1H.csv',
 './train_public/1h/L14.B01_1H.csv',
 './train_public/1h/L14.B02_1H.csv',
 './train_public/1h/L14.B03_1H.csv',
 './train_public/1h/L14.B04_1H.csv',
 './train_public/1h/L14.B05_1H.csv'
]

# Apply the cleaning function to each building dataset
for building_file in building_files:
    cleaned_file = clean_building_energy_data(building_file)
    print(f"Cleaned data saved to: {cleaned_file}")


Cleaned data saved to: ./train_public/1h/L03.B02_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L06.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L09.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L10.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B02_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B03_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B04_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B05_1H_cleaned.csv


In [ ]:
'./train_public/1h/L03.B02_1H.csv',
 './train_public/1h/L06.B01_1H.csv',
 './train_public/1h/L09.B01_1H.csv',
 './train_public/1h/L10.B01_1H.csv',
 './train_public/1h/L14.B01_1H.csv',
 './train_public/1h/L14.B02_1H.csv',
 './train_public/1h/L14.B03_1H.csv',
 './train_public/1h/L14.B04_1H.csv',
 './train_public/1h/L14.B05_1H.csv'

In [100]:
import pandas as pd
import os

# Function to clean a building energy consumption dataset
def clean_energy_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # 1. Check for daylight saving time duplicates in the "timestamp" column
    if data['timestamp'].duplicated().any():
        duplicates = data[data['timestamp'].duplicated()]
        print(f"Found duplicate timestamps in {file_path}:")
        print(duplicates)
    
    # 2. Split "timestamp" into "date" and "time"
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['date'] = data['timestamp'].dt.date
    data['time'] = data['timestamp'].dt.time

    # Drop the original "timestamp" column
    data = data.drop(columns=['timestamp'])

    # 3. Convert "main_meter(kW)" and "PV_battery_system(kW)" to numeric and handle missing values
    data['main_meter(kW)'] = pd.to_numeric(data['main_meter(kW)'], errors='coerce').fillna(0)

    if 'PV_battery_system(kW)' in data.columns:
        data['PV_battery_system(kW)'] = pd.to_numeric(data['PV_battery_system(kW)'], errors='coerce').fillna(0)

    # 4. Handle "energy_consumption(kW)" calculation and column renaming
    if 'PV_battery_system(kW)' in data.columns:
        data['energy_consumption(kW)'] = data['main_meter(kW)'] + data['PV_battery_system(kW)']
        data = data.drop(columns=['main_meter(kW)', 'PV_battery_system(kW)'])
    else:
        data = data.rename(columns={'main_meter(kW)': 'energy_consumption(kW)'})

    # Reorder columns to have "date", "time", and "energy_consumption(kW)"
    data = data[['date', 'time', 'energy_consumption(kW)']]

    # Save the cleaned data to a new CSV file
    cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')
    data.to_csv(cleaned_file_path, index=False)

    return cleaned_file_path

# Directory paths for input and output
input_dir = ''  # Replace with the path to your input directory

# List of input files
input_files = [
 './train_public/1h/L03.B02_1H.csv',
 './train_public/1h/L06.B01_1H.csv',
 './train_public/1h/L09.B01_1H.csv',
 './train_public/1h/L10.B01_1H.csv',
 './train_public/1h/L14.B01_1H.csv',
 './train_public/1h/L14.B02_1H.csv',
 './train_public/1h/L14.B03_1H.csv',
 './train_public/1h/L14.B04_1H.csv',
 './train_public/1h/L14.B05_1H.csv'
]

# Clean each file
for input_file in input_files:
    file_path = os.path.join(input_dir, input_file)
    cleaned_file = clean_energy_data(file_path)
    print(f"Cleaned data saved to: {cleaned_file}")


Cleaned data saved to: ./train_public/1h/L03.B02_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L06.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L09.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L10.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B01_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B02_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B03_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B04_1H_cleaned.csv
Cleaned data saved to: ./train_public/1h/L14.B05_1H_cleaned.csv


In [105]:
l06b01_1h_cleaned = pd.read_csv('train_public/1h_cleaned/L06.B01_1H_cleaned.csv')
l06b01_1h_cleaned.head()

,date,time,energy_consumption(kW)
0,2021-04-01,00:00:00,62.873325
1,2021-04-01,01:00:00,66.489815
2,2021-04-01,02:00:00,65.625000
3,2021-04-01,03:00:00,67.636673
4,2021-04-01,04:00:00,65.049164


In [106]:
l06b01_1h.head()

,timestamp,main_meter(kW),PV_battery_system(kW)
0,2021-04-01 00:00:00+00:00,62.873325,NaN
1,2021-04-01 01:00:00+00:00,66.489815,0.0
2,2021-04-01 02:00:00+00:00,65.625000,NaN
3,2021-04-01 03:00:00+00:00,67.636673,NaN
4,2021-04-01 04:00:00+00:00,65.049164,NaN


In [111]:
import pandas as pd
import numpy as np

# Function to clean weather data
def clean_weather_data(file_path, location):
    # Load the dataset
    data = pd.read_csv(file_path)

    # 1. Check for daylight saving time duplicates in the "timestamp" column
    if data['timestamp'].duplicated().any():
        duplicates = data[data['timestamp'].duplicated()]
        print(f"Found duplicate timestamps in {file_path}:")
        print(duplicates)
    else:
        print(f"No daylight saving recording in {file_path}")

    # 2. Split "timestamp" into "date" and "time"
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['date'] = data['timestamp'].dt.date
    data['time'] = data['timestamp'].dt.time

    # Drop the original "timestamp" column
    data = data.drop(columns=['timestamp'])

    # 3. Revise all negative values in "direct_solar_radiation(W/m^2)" and "diffuse_solar_radiation(W/m^2)" into 0
    data['direct_solar_radiation(W/m^2)'] = data['direct_solar_radiation(W/m^2)'].clip(lower=0)
    data['diffuse_solar_radiation(W/m^2)'] = data['diffuse_solar_radiation(W/m^2)'].clip(lower=0)

    # 4. Location-specific reasonable value checks

    # Temperature ranges for different locations
    temp_ranges = {
        'l03': (-10, 35),  # Denmark
        'l06': (-30, 30),  # Norway
        'l09': (-5, 45),   # Australia
        'l10': (-30, 30),  # Norway
        'l14': (-30, 30)   # Norway
    }
    temp_min, temp_max = temp_ranges[location]
    data['air_temperature_at_2m(deg_C)'] = data['air_temperature_at_2m(deg_C)'].clip(temp_min, temp_max)

    # Humidity should be between 0% and 100%
    data['relative_humidity_at_2m(%)'] = data['relative_humidity_at_2m(%)'].clip(0, 100)

    # Wind speed should not be negative; clip extreme wind speeds at 150 km/h
    data['wind_speed_at_10m(km/h)'] = data['wind_speed_at_10m(km/h)'].clip(0, 150)

    # 5. Convert columns to numeric and handle missing values
    columns_to_check = [
        'air_temperature_at_2m(deg_C)',
        'relative_humidity_at_2m(%)',
        'direct_solar_radiation(W/m^2)',
        'diffuse_solar_radiation(W/m^2)',
        'wind_speed_at_10m(km/h)',
        'wind_direction_at_10m(deg)'
    ]

    for column in columns_to_check:
        data[column] = pd.to_numeric(data[column], errors='coerce')

        # Fill missing values with the average of the adjacent values
        data[column] = data[column].interpolate(method='linear', limit_direction='both')

    # 6. Correct wind direction values
    data['wind_direction_at_10m(deg)'] = data['wind_direction_at_10m(deg)'].mod(360)

    # 7. Correct unrealistic solar radiation values
    data['direct_solar_radiation(W/m^2)'] = data['direct_solar_radiation(W/m^2)'].clip(upper=1000)
    data['diffuse_solar_radiation(W/m^2)'] = data['diffuse_solar_radiation(W/m^2)'].clip(upper=1000)

    # Save the cleaned data back to a new CSV file
    cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')
    data.to_csv(cleaned_file_path, index=False)

    return cleaned_file_path

# Directory paths for input and output
input_dir = 'train_public/weather/'  # Replace with the path to your weather data directory

# Location-specific file names
weather_files = {
    'l03': 'L03_weather_train.csv',
    'l06': 'L06_weather_train.csv',
    'l09': 'L09_weather_train.csv',
    'l10': 'L10_weather_train.csv',
    'l14': 'L14_weather_train.csv'
}

# Clean each weather file
for location, weather_file in weather_files.items():
    file_path = os.path.join(input_dir, weather_file)
    cleaned_file = clean_weather_data(file_path, location)
    print(f"Cleaned data for {location} saved to: {cleaned_file}")


No daylight saving recording in train_public/weather/L03_weather_train.csv
Cleaned data for l03 saved to: train_public/weather/L03_weather_train_cleaned.csv
No daylight saving recording in train_public/weather/L06_weather_train.csv
Cleaned data for l06 saved to: train_public/weather/L06_weather_train_cleaned.csv
No daylight saving recording in train_public/weather/L09_weather_train.csv
Cleaned data for l09 saved to: train_public/weather/L09_weather_train_cleaned.csv
No daylight saving recording in train_public/weather/L10_weather_train.csv
Cleaned data for l10 saved to: train_public/weather/L10_weather_train_cleaned.csv
No daylight saving recording in train_public/weather/L14_weather_train.csv
Cleaned data for l14 saved to: train_public/weather/L14_weather_train_cleaned.csv


In [135]:
# 30min data
l09b01_30min = pd.read_csv('train_public/30min/L09.B01_30min.csv')
l10b01_30min = pd.read_csv('train_public/30min/L10.B01_30min.csv')

# 15min data
l09b01_15min = pd.read_csv('train_public/15min/L09.B01_15min.csv')
l10b01_15min = pd.read_csv('train_public/15min/L10.B01_15min.csv')

# 5min data
l09b01_5min = pd.read_csv('train_public/5min/L09.B01_5min.csv')

print('l09b01_30min', l09b01_30min.shape)
print("l10b01_30min", l10b01_30min.shape)
print('l09b01_15min', l09b01_15min.shape)
print('l10b01_15min', l10b01_15min.shape)
print('l09b01_5min', l09b01_5min.shape)

l09b01_30min (16554, 3)
l10b01_30min (27731, 2)
l09b01_15min (33044, 3)
l10b01_15min (55460, 2)
l09b01_5min (98978, 3)


In [132]:
print(l09b01_5min.tail(10))
print(l09b01_15min.tail(10))
print(l09b01_30min.tail())
print(l09b01_1h.tail())

                       timestamp  main_meter(kW)  PV_battery_system(kW)
98968  2021-12-31 23:10:00+00:00           -42.0                  114.0
98969  2021-12-31 23:15:00+00:00           -70.0                  152.0
98970  2021-12-31 23:20:00+00:00          -147.0                  228.0
98971  2021-12-31 23:25:00+00:00           -17.0                   97.0
98972  2021-12-31 23:30:00+00:00            12.0                   62.0
98973  2021-12-31 23:35:00+00:00            12.0                   62.0
98974  2021-12-31 23:40:00+00:00            13.0                   65.0
98975  2021-12-31 23:45:00+00:00           -14.0                   97.0
98976  2021-12-31 23:50:00+00:00           -30.0                  114.0
98977  2021-12-31 23:55:00+00:00           -32.0                  112.0
                       timestamp  main_meter(kW)  PV_battery_system(kW)
33034  2021-12-31 21:30:00+00:00      -20.333333              87.000000
33035  2021-12-31 21:45:00+00:00      -13.000000              96

In [148]:
# Function to clean a building energy consumption dataset
def clean_energy_data(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # 1. Check for daylight saving time duplicates in the "timestamp" column
    if data['timestamp'].duplicated().any():
        duplicates = data[data['timestamp'].duplicated()]
        print(f"Found duplicate timestamps in {file_path}:")
        print(duplicates)
    else:
        print(f"No daylight saving recording in {file_path}")

    # 2. Split "timestamp" into "date" and "time"
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data['date'] = data['timestamp'].dt.date
    data['time'] = data['timestamp'].dt.time

    # Drop the original "timestamp" column
    data = data.drop(columns=['timestamp'])

    # 3. Convert "main_meter(kW)" and "PV_battery_system(kW)" to numeric and handle missing values
    data['main_meter(kW)'] = pd.to_numeric(data['main_meter(kW)'], errors='coerce').fillna(0)

    if 'PV_battery_system(kW)' in data.columns:
        data['PV_battery_system(kW)'] = pd.to_numeric(data['PV_battery_system(kW)'], errors='coerce').fillna(0)

    # 4. Handle "energy_consumption(kW)" calculation and column renaming
    if 'PV_battery_system(kW)' in data.columns:
        data['energy_consumption(kW)'] = data['main_meter(kW)'] + data['PV_battery_system(kW)']
        data = data.drop(columns=['main_meter(kW)', 'PV_battery_system(kW)'])
    else:
        data = data.rename(columns={'main_meter(kW)': 'energy_consumption(kW)'})

    # Reorder columns to have "date", "time", and "energy_consumption(kW)"
    data = data[['date', 'time', 'energy_consumption(kW)']]

    # Save the cleaned data to a new CSV file
    cleaned_file_path = file_path.replace('.csv', '_cleaned.csv')
    data.to_csv(cleaned_file_path, index=False)

    return cleaned_file_path

# Directory paths for input and output
input_dir = 'train_public/<1h/'  # Replace with the path to your input directory

# List of input files
input_files = [
    'L09.B01_30min.csv', 
    'l10.B01_30min.csv', 
    'L09.B01_15min.csv', 
    'L10.B01_15min.csv', 
    'L09.B01_5min.csv'
]

# Clean each file
for input_file in input_files:
    file_path = os.path.join(input_dir, input_file)
    cleaned_file = clean_energy_data(file_path)
    print(f"Cleaned data saved to: {cleaned_file}")

No daylight saving recording in train_public/<1h/L09.B01_30min.csv
Cleaned data saved to: train_public/<1h/L09.B01_30min_cleaned.csv
No daylight saving recording in train_public/<1h/l10.B01_30min.csv
Cleaned data saved to: train_public/<1h/l10.B01_30min_cleaned.csv
No daylight saving recording in train_public/<1h/L09.B01_15min.csv
Cleaned data saved to: train_public/<1h/L09.B01_15min_cleaned.csv
No daylight saving recording in train_public/<1h/L10.B01_15min.csv
Cleaned data saved to: train_public/<1h/L10.B01_15min_cleaned.csv
No daylight saving recording in train_public/<1h/L09.B01_5min.csv
Cleaned data saved to: train_public/<1h/L09.B01_5min_cleaned.csv


### Possible additional check
1. Timestamp Continuity Check:
Purpose: Ensure that there are no missing or irregular time intervals in the dataset. For example, in a dataset with a 30-minute resolution, the difference between consecutive timestamps should always be 30 minutes.
Fix: If gaps or irregularities are detected, you may want to interpolate or add the missing timestamps with appropriate values (e.g., by interpolating the energy consumption).
2. Outlier Detection in Energy Consumption:
Purpose: Identify and address extreme outliers in the energy consumption data, which could indicate sensor errors or other anomalies.
Fix: You can use statistical methods (e.g., z-scores, interquartile range) to detect outliers and decide whether to correct, remove, or further investigate them.
3. Logical Consistency Checks:
Purpose: Ensure that energy consumption values are within a realistic range for the building and time of year. For instance, energy consumption values should generally be non-negative and should not exceed plausible limits based on the building's size and usage patterns.
Fix: Clip or flag values that fall outside of expected ranges for further investigation.
4. Seasonal/Time-of-Day Analysis:
Purpose: Perform a sanity check by comparing energy consumption patterns across different times of the day, days of the week, or seasons. For example, a significant energy spike during the night might indicate a potential issue.
Fix: Investigate and potentially adjust any unexpected patterns.
5. Correlation Analysis:
Purpose: Check the correlation between the main_meter(kW) and PV_battery_system(kW) columns (if present). In a well-functioning system, these might have a predictable relationship, especially if the PV system is directly offsetting some of the main meter load.
Fix: If the correlation is unexpectedly low or high, it might warrant further investigation or corrections.

In [153]:
# Function to check timestamp continuity
def check_timestamp_continuity(file_path, time_resolution):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Convert date and time columns to a single datetime column
    data['datetime'] = pd.to_datetime(data['date'].astype(str) + ' ' + data['time'])
    
    # Define the expected time delta based on the resolution
    time_deltas = {
        '1H': pd.Timedelta(hours=1),
        '30min': pd.Timedelta(minutes=30),
        '15min': pd.Timedelta(minutes=15),
        '5min': pd.Timedelta(minutes=5)
    }
    
    expected_delta = time_deltas[time_resolution]
    
    # Calculate the actual difference between consecutive timestamps
    data['delta'] = data['datetime'].diff()
    
    # Find rows where the difference is not equal to the expected delta
    irregularities = data[data['delta'] != expected_delta]
    
    # If any irregularities found, print them along with adjacent rows
    if not irregularities.empty:
        print(f"Irregular timestamps found in {file_path}:")
        for idx in irregularities.index:
            start_idx = max(0, idx - 2)
            end_idx = min(len(data) - 2, idx + 2)
            print(data.loc[start_idx:end_idx, ['date', 'time', 'delta']])
    else:
        print(f"No irregular timestamps found in {file_path}")

# Function to detect outliers in energy consumption
def detect_outliers(file_path):
    # Load the dataset
    data = pd.read_csv(file_path)

    # Calculate the Z-score for the energy consumption column
    data['z_score'] = np.abs(stats.zscore(data['energy_consumption(kW)']))

    # Define an outlier as a value with a Z-score > 3
    outliers = data[data['z_score'] > 3]

    # Print outliers if found
    if not outliers.empty:
        print(f"Outliers detected in {file_path}:")
        print(outliers[['date', 'time', 'energy_consumption(kW)', 'z_score']])
    else:
        print(f"No outliers detected in {file_path}")

# Directory paths for input files
input_dir = 'train_public/cleaned/'  # Replace with the correct path

# Files and their time resolutions
files_with_resolutions = {
    'L03.B02_1H_cleaned.csv': '1H',
    'L06.B01_1H_cleaned.csv': '1H',
    'L09.B01_1H_cleaned.csv': '1H',
    'L10.B01_1H_cleaned.csv': '1H',
    'L14.B01_1H_cleaned.csv': '1H',
    'L14.B02_1H_cleaned.csv': '1H',
    'L14.B03_1H_cleaned.csv': '1H',
    'L14.B04_1H_cleaned.csv': '1H',
    'L14.B05_1H_cleaned.csv': '1H',
    'L09.B01_30min_cleaned.csv': '30min',
    'L10.B01_30min_cleaned.csv': '30min',
    'L09.B01_15min_cleaned.csv': '15min',
    'L10.B01_15min_cleaned.csv': '15min',
    'L09.B01_5min_cleaned.csv': '5min'
}

# Run checks for each file
for file_name, resolution in files_with_resolutions.items():
    file_path = os.path.join(input_dir, file_name)
    
    # Check timestamp continuity
    check_timestamp_continuity(file_path, resolution)
    
    # Detect outliers
    detect_outliers(file_path)


Irregular timestamps found in train_public/cleaned/L03.B02_1H_cleaned.csv:
         date      time           delta
0  2020-12-01  00:00:00             NaT
1  2020-12-01  01:00:00 0 days 01:00:00
2  2020-12-01  02:00:00 0 days 01:00:00
Outliers detected in train_public/cleaned/L03.B02_1H_cleaned.csv:
            date      time  energy_consumption(kW)   z_score
1016  2021-01-12  08:00:00              211.758621  3.370834
1041  2021-01-13  09:00:00              211.166672  3.356213
1088  2021-01-15  08:00:00              210.816086  3.347554
1160  2021-01-18  08:00:00              220.204437  3.579439
1352  2021-01-26  08:00:00              214.899231  3.448404
...          ...       ...                     ...       ...
8275  2021-11-10  19:00:00              197.809525  3.026301
8287  2021-11-11  07:00:00              238.903503  4.041292
8383  2021-11-15  07:00:00              226.410721  3.732730
8384  2021-11-15  08:00:00              231.380951  3.855491
8385  2021-11-15  09:00:00  

In [159]:
l09b01_1H_cleaned = pd.read_csv('train_public/cleaned/L09.B01_1H_cleaned.csv')
l09b01_1H_cleaned[170:190]

,date,time,energy_consumption(kW)
170,2021-01-08,02:00:00,169.916667
171,2021-01-08,03:00:00,189.833333
172,2021-01-08,04:00:00,159.916667
173,2021-01-08,05:00:00,152.250000
174,2021-01-08,06:00:00,161.083333
175,2021-01-08,07:00:00,113.083333
176,2021-01-08,08:00:00,117.250000
177,2021-01-08,09:00:00,117.750000
178,2021-01-08,10:00:00,116.916667
179,2021-01-08,11:00:00,117.000000


In [162]:
l09b01_1h[170:190]

,timestamp,main_meter(kW),PV_battery_system(kW)
170,2021-01-08 02:00:00+00:00,154.916667,15.0
171,2021-01-08 03:00:00+00:00,174.833333,15.0
172,2021-01-08 04:00:00+00:00,144.916667,15.0
173,2021-01-08 05:00:00+00:00,137.250000,15.0
174,2021-01-08 06:00:00+00:00,146.083333,15.0
175,2021-01-08 07:00:00+00:00,98.083333,15.0
176,2021-01-08 08:00:00+00:00,102.250000,15.0
177,2021-01-08 09:00:00+00:00,102.750000,15.0
178,2021-01-08 10:00:00+00:00,101.916667,15.0
179,2021-01-08 11:00:00+00:00,102.000000,15.0


In [165]:
l09b01_1h.shape

(8287, 3)

In [ ]:
# Directory paths for input files
input_energy_dir = 'train_public/cleaned/'  # Replace with the correct path
input_weather_dir = 'train_public/weather_cleaned'  # Replace with the correct path

In [178]:
import pandas as pd
import os

# Function to match and concatenate energy data with weather data
def match_and_concatenate(energy_file, weather_file, time_resolution):
    # Load the building energy and weather datasets
    energy_data = pd.read_csv(energy_file)
    weather_data = pd.read_csv(weather_file)

    # Convert date and time columns to datetime
    energy_data['datetime'] = pd.to_datetime(energy_data['date'] + ' ' + energy_data['time'])
    weather_data['datetime'] = pd.to_datetime(weather_data['date'] + ' ' + weather_data['time'])

    if time_resolution == '1H':
        # 1. Concatenate for 1H time resolution
        merged_data = pd.merge(energy_data, weather_data, on=['datetime'], how='left')
        dropped_rows = merged_data[merged_data.isnull().any(axis=1)]
        if not dropped_rows.empty:
            print(f"Dropped rows in {energy_file}:")
            print(dropped_rows[['date', 'time']])
        merged_data.dropna(inplace=True)
    
    else:
        # Interpolation for 30min, 15min, and 5min resolutions
        if time_resolution == '30min':
            freq = '30min'
        elif time_resolution == '15min':
            freq = '15min'
        elif time_resolution == '5min':
            freq = '5min'
        
        # Interpolate weather data
        weather_data.set_index('datetime', inplace=True)
        weather_data_interpolated = weather_data.resample(freq).interpolate(method='linear').reset_index()
        
        # Merge with energy data
        merged_data = pd.merge(energy_data, weather_data_interpolated, on=['datetime'], how='left')
        print(f"Shape of concatenated dataset for {energy_file}: {merged_data.shape}")
    
    # Drop the datetime column as it's no longer needed after the merge
    merged_data.drop(columns=['datetime'], inplace=True)

    return merged_data

# Directory paths for input files
input_energy_dir = 'train_public/cleaned/'  # Replace with the correct path
input_weather_dir = 'train_public/weather_cleaned/'  # Replace with the correct path

# Location and time resolution based files
files_with_resolutions = {
    'L03.B02_1H_cleaned.csv': '1H',
    'L06.B01_1H_cleaned.csv': '1H',
    'L09.B01_1H_cleaned.csv': '1H',
    'L10.B01_1H_cleaned.csv': '1H',
    'L14.B01_1H_cleaned.csv': '1H',
    'L14.B02_1H_cleaned.csv': '1H',
    'L14.B03_1H_cleaned.csv': '1H',
    'L14.B04_1H_cleaned.csv': '1H',
    'L14.B05_1H_cleaned.csv': '1H',
    'L09.B01_30min_cleaned.csv': '30min',
    'L10.B01_30min_cleaned.csv': '30min',
    'L09.B01_15min_cleaned.csv': '15min',
    'L10.B01_15min_cleaned.csv': '15min',
    'L09.B01_5min_cleaned.csv': '5min'
}

# Match location to weather file
location_to_weather = {
    'L03': 'L03_weather_train_cleaned.csv',
    'L06': 'L06_weather_train_cleaned.csv',
    'L09': 'L09_weather_train_cleaned.csv',
    'L10': 'L10_weather_train_cleaned.csv',
    'L14': 'L14_weather_train_cleaned.csv'
}

# Run the match and concatenate process for each file
for energy_file, time_resolution in files_with_resolutions.items():
    # Extract the location from the file name
    location = energy_file.split('.')[0]
    
    # Get the corresponding weather file
    weather_file = location_to_weather[location]
    
    # Full paths to the files
    energy_file_path = os.path.join(input_energy_dir, energy_file)
    weather_file_path = os.path.join(input_weather_dir, weather_file)
    
    # Perform the match and concatenate
    concatenated_data = match_and_concatenate(energy_file_path, weather_file_path, time_resolution)
    
    # Save the concatenated data to a new file
    output_file = energy_file.replace('_cleaned.csv', '_with_weather.csv')
    concatenated_data.to_csv(os.path.join(input_energy_dir, output_file), index=False)
    print(f"Concatenated dataset saved to: {output_file}")


Concatenated dataset saved to: L03.B02_1H_with_weather.csv
Concatenated dataset saved to: L06.B01_1H_with_weather.csv
Concatenated dataset saved to: L09.B01_1H_with_weather.csv
Concatenated dataset saved to: L10.B01_1H_with_weather.csv
Concatenated dataset saved to: L14.B01_1H_with_weather.csv
Concatenated dataset saved to: L14.B02_1H_with_weather.csv
Concatenated dataset saved to: L14.B03_1H_with_weather.csv
Concatenated dataset saved to: L14.B04_1H_with_weather.csv
Concatenated dataset saved to: L14.B05_1H_with_weather.csv
Shape of concatenated dataset for train_public/cleaned/L09.B01_30min_cleaned.csv: (16554, 12)
Concatenated dataset saved to: L09.B01_30min_with_weather.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/2151781911.py:34: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data_interpolated = weather_data.resample(freq).interpolate(method='linear').reset_index()
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/2151781911.py:34: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data_interpolated = weather_data.resample(freq).interpolate(method='linear').reset_index()


Shape of concatenated dataset for train_public/cleaned/L10.B01_30min_cleaned.csv: (27731, 12)
Concatenated dataset saved to: L10.B01_30min_with_weather.csv
Shape of concatenated dataset for train_public/cleaned/L09.B01_15min_cleaned.csv: (33044, 12)


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/2151781911.py:34: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data_interpolated = weather_data.resample(freq).interpolate(method='linear').reset_index()


Concatenated dataset saved to: L09.B01_15min_with_weather.csv
Shape of concatenated dataset for train_public/cleaned/L10.B01_15min_cleaned.csv: (55460, 12)


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/2151781911.py:34: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data_interpolated = weather_data.resample(freq).interpolate(method='linear').reset_index()


Concatenated dataset saved to: L10.B01_15min_with_weather.csv
Shape of concatenated dataset for train_public/cleaned/L09.B01_5min_cleaned.csv: (98978, 12)


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/2151781911.py:34: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  weather_data_interpolated = weather_data.resample(freq).interpolate(method='linear').reset_index()


Concatenated dataset saved to: L09.B01_5min_with_weather.csv


In [179]:
# Function to process the concatenated files
def process_concatenated_file(file_path, output_dir):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Drop the 'date_y' and 'time_y' columns
    data = data.drop(columns=['date_y', 'time_y'])
    
    # Rename 'date_x' and 'time_x' to 'date' and 'time'
    data = data.rename(columns={'date_x': 'date', 'time_x': 'time'})
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save the processed data to the output directory
    output_file = os.path.join(output_dir, os.path.basename(file_path))
    data.to_csv(output_file, index=False)
    print(f"Processed file saved to: {output_file}")

# Directory paths
input_dir = 'train_public/concatenated/'  # Replace with the correct path
output_dir = 'train_public/pre_trained/'  # Replace with the correct path

# List of concatenated files to process
concatenated_files = [
    'L03.B02_1H_with_weather.csv',
    'L10.B01_1H_with_weather.csv',
    'L06.B01_1H_with_weather.csv',
    'L10.B01_30min_with_weather.csv',
    'L09.B01_15min_with_weather.csv',
    'L14.B01_1H_with_weather.csv',
    'L09.B01_1H_with_weather.csv',
    'L14.B02_1H_with_weather.csv',
    'L09.B01_30min_with_weather.csv',
    'L14.B03_1H_with_weather.csv',
    'L09.B01_5min_with_weather.csv',
    'L14.B04_1H_with_weather.csv',
    'L10.B01_15min_with_weather.csv',
    'L14.B05_1H_with_weather.csv'
]

# Process each file
for file_name in concatenated_files:
    file_path = os.path.join(input_dir, file_name)
    process_concatenated_file(file_path, output_dir)


Processed file saved to: train_public/pre_trained/L03.B02_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L10.B01_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L06.B01_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L10.B01_30min_with_weather.csv
Processed file saved to: train_public/pre_trained/L09.B01_15min_with_weather.csv
Processed file saved to: train_public/pre_trained/L14.B01_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L09.B01_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L14.B02_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L09.B01_30min_with_weather.csv
Processed file saved to: train_public/pre_trained/L14.B03_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L09.B01_5min_with_weather.csv
Processed file saved to: train_public/pre_trained/L14.B04_1H_with_weather.csv
Processed file saved to: train_public/pre_trained/L10

In [181]:
# Function to assign a season based on the date
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

# Function to perform NMF and save the disaggregated heating/cooling load
def disaggregate_heating_cooling(file_path, output_dir):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Add a season column based on the month
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'])
    data['season'] = data['datetime'].dt.month.apply(get_season)
    
    # Enhance the feature matrix for NMF by including additional relevant variables
    features = ['energy_consumption(kW)', 'air_temperature_at_2m(deg_C)', 
                'relative_humidity_at_2m(%)', 'direct_solar_radiation(W/m^2)', 
                'diffuse_solar_radiation(W/m^2)', 'wind_speed_at_10m(km/h)', 
                'wind_direction_at_10m(deg)']
    
    # Clip negative values to 0 to meet NMF requirements
    data[features] = data[features].clip(lower=0)
    
    # Apply NMF separately for each season
    results = []
    for season in ['winter', 'spring', 'summer', 'fall']:
        season_data = data[data['season'] == season]
        if season_data.empty:
            continue
        
        # Prepare the feature matrix for NMF
        X = season_data[features]
        
        # Apply NMF
        nmf = NMF(n_components=2, init='random', random_state=42, max_iter=1000)
        W = nmf.fit_transform(X)
        H = nmf.components_
        
        # Assume the first component (W[:, 0]) corresponds to the temperature-dependent load
        season_data['Temperature_dependent(kW)'] = W[:, 0]
        results.append(season_data[['datetime', 'Temperature_dependent(kW)']])
    
    # Concatenate the results for all seasons
    final_result = pd.concat(results)
    
    # Create the timestamp column
    final_result['timestamp'] = final_result['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S+00:00')
    
    # Select the relevant columns for output
    final_output = final_result[['timestamp', 'Temperature_dependent(kW)']]
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save the result to a new CSV file
    output_file = os.path.join(output_dir, os.path.basename(file_path).replace('_with_weather.csv', '_disaggregated.csv'))
    final_output.to_csv(output_file, index=False)
    print(f"Disaggregated file saved to: {output_file}")

# Directory paths
input_dir = 'train_public/pre_trained/'  # Replace with the correct path
output_dir = 'results/'  # Replace with the correct path

# List of cleaned files to process
cleaned_files = [
    'L03.B02_1H_with_weather.csv',
    'L10.B01_1H_with_weather.csv',
    'L06.B01_1H_with_weather.csv',
    'L10.B01_30min_with_weather.csv',
    'L09.B01_15min_with_weather.csv',
    'L14.B01_1H_with_weather.csv',
    'L09.B01_1H_with_weather.csv',
    'L14.B02_1H_with_weather.csv',
    'L09.B01_30min_with_weather.csv',
    'L14.B03_1H_with_weather.csv',
    'L09.B01_5min_with_weather.csv',
    'L14.B04_1H_with_weather.csv',
    'L10.B01_15min_with_weather.csv',
    'L14.B05_1H_with_weather.csv'
]

# Perform NMF for each file
for file_name in cleaned_files:
    file_path = os.path.join(input_dir, file_name)
    disaggregate_heating_cooling(file_path, output_dir)


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Disaggregated file saved to: results/L03.B02_1H_disaggregated.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Disaggregated file saved to: results/L10.B01_1H_disaggregated.csv
Disaggregated file saved to: results/L06.B01_1H_disaggregated.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/1332892890.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]


ValueError: Input X contains NaN.
NMF does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [182]:
import pandas as pd
from sklearn.decomposition import NMF
import os

# Function to assign a season based on the date
def get_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'

# Function to perform NMF and save the disaggregated heating/cooling load
def disaggregate_heating_cooling(file_path, output_dir):
    # Load the dataset
    data = pd.read_csv(file_path)
    
    # Add a season column based on the month
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'])
    data['season'] = data['datetime'].dt.month.apply(get_season)
    
    # Enhance the feature matrix for NMF by including additional relevant variables
    features = ['energy_consumption(kW)', 'air_temperature_at_2m(deg_C)', 
                'relative_humidity_at_2m(%)', 'direct_solar_radiation(W/m^2)', 
                'diffuse_solar_radiation(W/m^2)', 'wind_speed_at_10m(km/h)', 
                'wind_direction_at_10m(deg)']
    
    # Clip negative values to 0 to meet NMF requirements
    data[features] = data[features].clip(lower=0)
    
    # Handle missing values by using linear interpolation
    data[features] = data[features].interpolate(method='linear', limit_direction='both')
    
    # Apply NMF separately for each season
    results = []
    for season in ['winter', 'spring', 'summer', 'fall']:
        season_data = data[data['season'] == season]
        if season_data.empty:
            continue
        
        # Prepare the feature matrix for NMF
        X = season_data[features]
        
        # Apply NMF
        nmf = NMF(n_components=2, init='random', random_state=42, max_iter=1000)
        W = nmf.fit_transform(X)
        H = nmf.components_
        
        # Assume the first component (W[:, 0]) corresponds to the temperature-dependent load
        season_data['Temperature_dependent(kW)'] = W[:, 0]
        results.append(season_data[['datetime', 'Temperature_dependent(kW)']])
    
    # Concatenate the results for all seasons
    final_result = pd.concat(results)
    
    # Create the timestamp column
    final_result['timestamp'] = final_result['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S+00:00')
    
    # Select the relevant columns for output
    final_output = final_result[['timestamp', 'Temperature_dependent(kW)']]
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save the result to a new CSV file
    output_file = os.path.join(output_dir, os.path.basename(file_path).replace('_with_weather.csv', '_disaggregated.csv'))
    final_output.to_csv(output_file, index=False)
    print(f"Disaggregated file saved to: {output_file}")

# Directory paths
input_dir = 'train_public/pre_trained/'  # Replace with the correct path
output_dir = 'results/'  # Replace with the correct path

# List of cleaned files to process
cleaned_files = [
    'L03.B02_1H_with_weather.csv',
    'L10.B01_1H_with_weather.csv',
    'L06.B01_1H_with_weather.csv',
    'L10.B01_30min_with_weather.csv',
    'L09.B01_15min_with_weather.csv',
    'L14.B01_1H_with_weather.csv',
    'L09.B01_1H_with_weather.csv',
    'L14.B02_1H_with_weather.csv',
    'L09.B01_30min_with_weather.csv',
    'L14.B03_1H_with_weather.csv',
    'L09.B01_5min_with_weather.csv',
    'L14.B04_1H_with_weather.csv',
    'L10.B01_15min_with_weather.csv',
    'L14.B05_1H_with_weather.csv'
]

# Perform NMF for each file
for file_name in cleaned_files:
    file_path = os.path.join(input_dir, file_name)
    disaggregate_heating_cooling(file_path, output_dir)


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Disaggregated file saved to: results/L03.B02_1H_disaggregated.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Disaggregated file saved to: results/L10.B01_1H_disaggregated.csv
Disaggregated file saved to: results/L06.B01_1H_disaggregated.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

...

/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Disaggregated file saved to: results/L09.B01_5min_disaggregated.csv
Disaggregated file saved to: results/L14.B04_1H_disaggregated.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Disaggregated file saved to: results/L10.B01_15min_disaggregated.csv
Disaggregated file saved to: results/L14.B05_1H_disaggregated.csv


/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Temperature_dependent(kW)'] = W[:, 0]
/var/folders/wj/rnnxbxj55kn0jfp29lqz0_k00000gp/T/ipykernel_67220/3478108615.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s